In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.porter import PorterStemmer

In [2]:
def indiscernibility(matrix,y):
    df=pd.DataFrame(matrix)
    # this will return the list of columns     
    y=list(df.columns)
    grouped_df=df.groupby(y)
    
    ind_R=list(list())
    for key, item in grouped_df:
#         print(grouped_df.get_group(key), "\n",grouped_df.get_group(key).index ,"\n\n")
        lis=[]
        for i in grouped_df.get_group(key).index:
            lis.append(i)
        ind_R.append(list(lis))
    return ind_R

def encoding_discourse_type(x):
    if x=="Lead":
        return 0
    if x=="Position":
        return 1
    if x=="Evidence":
        return 2
    if x=="Claim":
        return 3
    if x=="Concluding Statement":
        return 4
    if x=="Counterclaim":
        return 5
    if x=='Rebuttal':
        return 6
    
def stemming_stopwords_removing(df):
    corpus=[]
    for i in range(len(df)):
        review=re.sub('[^a-zA-Z]',' ',df["discourse_text"][i])
        review=review.lower()
        review=review.split()
        ps=PorterStemmer()
        all_stopwords=stopwords.words("english")
        review=[ps.stem(word) for word in review if not word in set(all_stopwords)]
        review=' '.join(review)
        corpus.append(review)
    return corpus

# storing the total occurrence.......
def get_total_index_words(corpus):
    index_word={}
    for i in corpus:
        s=i.split()
        for j in s:
            if j not in index_word:
                index_word[j]=1
            else:
                index_word[j]+=1
    return index_word
    
def get_values(dataset,threshold=1):
    
    # taking sample of 20 documents for lead category....
    df = dataset

    total_corpus = stemming_stopwords_removing(df)
    # print(total_corpus)

    # getting total index words and their count in the taken sample as a dict
    total_index_words = get_total_index_words(total_corpus)
    # print(len(lead_index_words))

    # Creating a list of total keywords before filtering..
    total_keywords = list(total_index_words.keys())

    # Creating a matrix of width equals len(lead_keywords)
    matrix=np.zeros((len(df),len(total_keywords)))

    
    # Storing occurrence of each term in each document respectively
    for i in range(len(total_corpus)):
        s = total_corpus[i].split()
        for h in s:
            j = total_keywords.index(h)
            matrix[i,j] += 1


    # Storing their weights....
    weighted_matrix = np.copy(matrix)
    
    for i in range(len(df)):
        for j in range(len(total_keywords)):
            weighted_matrix[i,j] = weighted_matrix[i,j] / total_index_words[total_keywords[j]]
    #            print(weighted_matrix[i,j])



    # FILTERING WEIGHTS with a threshold.......
    valid_index = []
    for i in range(len(df)):
        for j in range(len(total_keywords)):
            if weighted_matrix[i,j] >= threshold:
                valid_index.append(j)

    # removing duplicates and storing them in a list.......    
    valid_index = list(set(valid_index))


    # # Storing the final keywords.... 
    valid_index_words = []
    for i in range(len(valid_index)):
        valid_index_words.append(total_keywords[valid_index[i]])
    # print(valid_lead_index_words)

    return total_keywords,total_index_words,matrix, weighted_matrix, valid_index, valid_index_words

def get_test_matrix(dataset,total_sample_keywords,threshold=1):
      # taking sample of 20 documents for lead category....
    df = dataset

    total_corpus = stemming_stopwords_removing(df)
    # print(total_corpus)

    # getting total index words and their count in the taken sample as a dict
    total_index_words = get_total_index_words(total_corpus)


    # Creating a list of total keywords before filtering..
    total_keywords = total_sample_keywords
    # print(lead_keywords)

    # Creating a matrix of width equals len(lead_keywords)
    matrix=np.zeros((len(df),len(total_keywords)))
    print(len(df),len(total_keywords))

    
    # Storing occurrence of each term in each document respectively
    for i in range(len(total_corpus)):
        s = total_corpus[i].split()
        for h in s:
            try:
                j = total_keywords.index(h)
                matrix[i,j] += 1
            except:
                continue
            
    # Storing their weights....
    weighted_matrix = np.copy(matrix)
    
    for i in range(len(df)):
        for j in range(len(total_keywords)):
            try:
                weighted_matrix[i,j] = weighted_matrix[i,j] / total_index_words[total_keywords[j]]
            except:
                continue


    # FILTERING WEIGHTS with a threshold.......
    valid_index = []
    for i in range(len(df)):
        for j in range(len(total_keywords)):
            if weighted_matrix[i,j] >= threshold:
                valid_index.append(j)

    # removing duplicates and storing them in a list.......    
    valid_index = list(set(valid_index))
    


    # # Storing the final keywords.... 
    valid_index_words = []
    for i in range(len(valid_index)):
        valid_index_words.append(total_keywords[valid_index[i]])
    # print(valid_lead_index_words)

    return total_keywords,total_index_words,matrix, weighted_matrix, valid_index, valid_index_words


In [3]:
def single_element_indiscernibility(matrix,index):
    df=pd.DataFrame(matrix)
    # this will return the list of columns     
    y=list(df.columns)
    
    grouped_df=df.groupby(y[index])
    
    ind_R=list(list())
    for key, item in grouped_df:
    #print(grouped_df.get_group(key), "\n",grouped_df.get_group(key).index ,"\n\n")
        lis=[]
        for i in grouped_df.get_group(key).index:
            lis.append(i)
        ind_R.append(list(lis))
    return ind_R

def indiscernibility(matrix,y):
    if len(y)==0:
        return []
    df=pd.DataFrame(matrix)
    # this will return the list of columns     
    grouped_df=df.groupby(y)
    ind_R=list(list())
    for key, item in grouped_df:
    #print(grouped_df.get_group(key), "\n",grouped_df.get_group(key).index ,"\n\n")
        lis=[]
        for i in grouped_df.get_group(key).index:
            lis.append(i)
        ind_R.append(list(lis))
    return ind_R

    

In [4]:
train=pd.read_csv("train.csv")
train_head=train.head(100)
train_mid=train[300:400]
train_tail=train.tail(100)
train=pd.concat([train_head,train_tail,train_mid])
train.reset_index(inplace=True)
train

,index,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
2,2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
3,3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...
4,4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...
...,...,...,...,...,...,...,...,...,...
295,395,E527586F851C,1.622749e+12,1305.0,1334.0,We should not text and drive.,Position,Position 1,226 227 228 229 230 231
296,396,E527586F851C,1.622749e+12,1335.0,1614.0,It is overall a distraction and could lead you...,Concluding Statement,Concluding Statement 1,232 233 234 235 236 237 238 239 240 241 242 24...
297,397,5F1CF4B91975,1.622748e+12,0.0,88.0,Drivers should not be able to use cell phones ...,Position,Position 1,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
298,398,5F1CF4B91975,1.622748e+12,89.0,211.0,"Every day, people die in car accidents. Studie...",Claim,Claim 1,16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 3...


In [5]:
# position=train[train["discourse_type"]=="Position"].copy()
# position.reset_index(inplace=True)

# evidence=train[train["discourse_type"]=="Evidence"].copy()
# evidence.reset_index(inplace=True)

# claim=train[train["discourse_type"]=="Claim"].copy()
# claim.reset_index(inplace=True)

# counter_claim=train[train["discourse_type"]=="Counterclaim"].copy()
# counter_claim.reset_index(inplace=True)

# rebuttal=train[train["discourse_type"]=="Rebuttal"].copy()
# rebuttal.reset_index(inplace=True)

# lead=train[train["discourse_type"]=="Lead"].copy()
# lead.reset_index(inplace=True)

# concluding_statement=train[train["discourse_type"]=="Concluding Statement"].copy()
# concluding_statement.reset_index(inplace=True)

In [6]:
# t=position.head(50)+evidence.head(50)+claim.head(50)+counter_claim.head(50)+rebuttal.head(50)+lead.head(50)+concluding_statement.head(50

In [7]:
# Now i will send my training data to get its list of final keywords after eliminating useles words from our list
# using weighted matrix .....

train_total_keywords,train_total_index_words, train_matrix, train_weighted_matrix, train_valid_index, train_final_keywords=get_values(train)

In [8]:
print(len(train_weighted_matrix))
c_attr=train_weighted_matrix.copy()

300


In [9]:
train["Y"]=train["discourse_type"].copy()
train["Y"]=train["Y"].apply(lambda x: encoding_discourse_type(x))
y=train["Y"].values
print(len(y))
dec_attr=y.copy()

300


In [10]:
# NOW REMOVING IDENTICAL COLUMNS FROM OUR MATRIX INPLACE.......
print((c_attr))
print("shape of old training attribute",c_attr.shape)
grp=np.column_stack(c_attr)
temp=pd.DataFrame(grp)
# print(temp.shape)

temp_grp=indiscernibility(temp,list(temp.columns))
# print(temp_grp)

unique_columns=[]
for i in range(len(temp_grp)):
    unique_columns.append(temp_grp[i][0])
    
unique_columns.sort()
print()
# print("Unique columns:",unique_columns)
print("\nNo of unique columns:",len(unique_columns))

unique_col_table=list(list())
for i in unique_columns:
    unique_col_table.append(list(grp[i]))
unique_col_table=np.array(unique_col_table)
# print("\nunique_col_table:")
print(unique_col_table)
print("\n unique_col_table shape",unique_col_table.shape)

new_train_attributes=np.column_stack(unique_col_table)

print("\nnew train attributes after preserving only one repetitve element:\n",new_train_attributes)
print("\nshape of new train attributes:",new_train_attributes.shape)
c_attr=new_train_attributes.copy()

repetitive_columns=list(set(list(range(len(c_attr[0]))))-set(unique_columns))
print("number of repeated columns are: ", len(repetitive_columns))

[[1.         0.25       0.11111111 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
shape of old training attribute (300, 1192)


No of unique columns: 777
[[1.         0.         0.         ... 0.         0.         0.        ]
 [0.25       0.         0.         ... 0.         0.         0.        ]
 [0.11111111 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]]

 unique_col_table shape (777, 300)

new

In [11]:
def calculate_LA(R_e,ind):
    nR_e=list(set())
    nind=list(set())
    ans=[]
    for i in R_e:
        nR_e.append(set(i))
    for i in ind:
        nind.append(set(i))
    
    for i in range(len(nind)):
        for j in range(len(nR_e)):
            if nind[i]&nR_e[j]==nind[i]:
                for val in nind[i]:
                    ans.append(val)
    ans=list(set(ans))
    return ans

def calculate_dependency(R_e,ind,universe=list(range(len(dec_attr)))):
     # here universe is set of all objects or terms.
    if len(ind)==0:
        return 0
    ans=calculate_LA(R_e,ind)
    return len(ans)/len( universe)
                

In [12]:
combined_attr=c_attr.copy()
new_arr=np.array([])
for i in range(len(combined_attr)):
    new_arr=np.append(new_arr,np.append(combined_attr[i],dec_attr[i]))
# now reshape it...
new_arr=new_arr.reshape(len(dec_attr),len(c_attr[0])+1)
combined_attr=new_arr.copy()
combined_attr
# df1=pd.DataFrame(combined_attr)
# print(list(df1.columns))

array([[1.        , 0.25      , 0.11111111, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        2.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        3.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        2.        ]])

In [13]:
R_e=single_element_indiscernibility(combined_attr,len(combined_attr[0])-1)
# R_e

In [14]:
# Feature selection through QUICK REDUCT ALGO
C=[]
for i in range(len(c_attr[0])):
    C.append(i)
C=set(C)
R=set()
T=set()

dep_R=0
dep_T=0
dep_Re=calculate_dependency(R_e,indiscernibility(c_attr,list(C)))
reduct=True
while dep_R!=1:
    T=R.copy()
    for x in C-R:
        dep_R=calculate_dependency(R_e,indiscernibility(c_attr,list(R.union({x}))))
        if dep_R>dep_T:
            T=R.union({x}).copy()
            dep_T=calculate_dependency(R_e,indiscernibility(c_attr,list(T)))
    if len(R)==len(T):
        reduct=False
        break
    R=T.copy()
    
    print(R)
    dep_R=calculate_dependency(R_e,indiscernibility(c_attr,list(R)))
    
print(R)
print(calculate_dependency(R_e,indiscernibility(c_attr,list(R))))

# computing new reduced matrix
print("Old conditional matrix before reducing:\n",c_attr)
new_matrix=c_attr[:,list(R)]
c_attr=new_matrix.copy()
print("\nNew conditional matrix after reducing:\n",new_matrix)
# Note: decision attribute will remain the same
print("\nDecision matrix is:\n",dec_attr)
# computing new combined matrix

# combined_attr=c_attr.copy()
# new_arr=np.array([])
# for i in range(len(combined_attr)):
#     new_arr=np.append(new_arr,np.append(combined_attr[i],dec_attr[i]))
# # now reshape it...
# new_arr=new_arr.reshape(len(dec_attr),len(c_attr[0])+1)
# combined_attr=new_arr.copy()
# print("\nNew combined matrix:\n",combined_attr)

# Storing final reduced columns list
reduced_col=list(R)

# storing new_terms
# new_terms=[]
# for i in reduced_col:
#     new_terms.append(terms[i])
# print("\nNew columns as per order:\n",new_terms)
    




{72}
{72, 43}
{72, 43, 148}
{72, 43, 148, 4}
{148, 4, 72, 105, 43}
{148, 4, 72, 105, 43, 15}
{80, 148, 4, 72, 105, 43, 15}
{4, 72, 105, 43, 524, 15, 80, 148}
{4, 72, 105, 43, 524, 15, 80, 148, 29}
{4, 519, 72, 105, 43, 524, 15, 80, 148, 29}
{4, 519, 72, 105, 43, 524, 15, 80, 148, 119, 29}
{4, 102, 519, 72, 105, 43, 524, 15, 80, 148, 119, 29}
{4, 102, 519, 72, 105, 43, 524, 15, 80, 148, 119, 29, 126}
{4, 102, 519, 72, 105, 43, 524, 15, 80, 50, 148, 119, 29, 126}
{4, 102, 519, 72, 105, 43, 524, 15, 80, 50, 148, 52, 119, 29, 126}
{4, 519, 72, 524, 15, 80, 148, 88, 29, 102, 105, 43, 50, 52, 119, 126}
{4, 519, 72, 524, 15, 80, 207, 148, 88, 29, 102, 105, 43, 50, 52, 119, 126}
{4, 519, 72, 524, 15, 80, 207, 148, 88, 29, 102, 105, 43, 50, 51, 52, 119, 126}
{4, 519, 72, 524, 15, 80, 207, 148, 88, 29, 102, 105, 43, 50, 51, 52, 119, 59, 126}
{4, 519, 72, 524, 15, 80, 207, 148, 21, 88, 29, 102, 105, 43, 50, 51, 52, 119, 59, 126}
{4, 519, 72, 524, 15, 80, 207, 148, 21, 88, 29, 102, 105, 43, 50, 51

In [15]:
print(c_attr)
    

[[0.00816327 0.         0.01388889 ... 0.         0.         0.        ]
 [0.00408163 0.         0.         ... 0.         0.         0.        ]
 [0.00408163 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.00408163 0.         0.         ... 0.         0.         0.        ]
 [0.00408163 0.         0.         ... 0.         0.         0.        ]
 [0.00408163 0.         0.         ... 0.         0.         0.        ]]


In [16]:
from sklearn import datasets
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from frlearn.base import probabilities_from_scores, select_class
from frlearn.classifiers import FRNN
from frlearn.feature_preprocessors import RangeNormaliser

In [17]:
# Split into train and test sets.
X_train, X_test, y_train, y_test = train_test_split(c_attr, y, stratify=y, random_state=0)

In [18]:
X_train.shape

(225, 39)

In [19]:
y_train.shape

(225,)

In [20]:
# Create an instance of the FRNN classifier, construct the model, and query on the test set.
clf = FRNN(preprocessors=(RangeNormaliser(), ))
model = clf(X_train, y_train)
scores = model(X_test)

In [21]:
# Convert scores to probabilities and calculate the AUROC.
probabilities = probabilities_from_scores(scores)
auroc = roc_auc_score(y_test, probabilities, multi_class='ovo')
print('AUROC:', auroc)

# Select classes with the highest scores and calculate the accuracy.
classes = select_class(scores)
accuracy = accuracy_score(y_test, classes)
print('accuracy:', accuracy)

AUROC: 0.6735270127911606
accuracy: 0.4266666666666667
